In [1]:
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import numpy as np
#from autoaugment import ImageNetPolicy
from torchvision import datasets, transforms, models

In [2]:
#running on GPU
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [3]:
import os
from collections import defaultdict
import os
import collections
from os import listdir
from os.path import isfile, join
import shutil
import stat

In [4]:
import multiprocessing as mp

num_processes = 6
pool = mp.Pool(processes=num_processes)

In [5]:
#generating train and test class folders
def generate_dir_file_map(path):
    dir_files = defaultdict(list)
    with open(path, 'r') as txt:
        files = [l.strip() for l in txt.readlines()]
        for f in files:
            dir_name, id = f.split('/')
            dir_files[dir_name].append(id + '.jpg')
    return dir_files

train_dir_files = generate_dir_file_map('../input/food41/meta/meta/train.txt')
test_dir_files = generate_dir_file_map('../input/food41/meta/meta/test.txt')


def ignore_train(d, filenames):
    print(d)
    subdir = d.split('/')[-1]
    to_ignore = train_dir_files[subdir]
    return to_ignore

def ignore_test(d, filenames):
    print(d)
    subdir = d.split('/')[-1]
    to_ignore = test_dir_files[subdir]
    return to_ignore

In [6]:
class_to_ix = {}
ix_to_class = {}
with open('../input/food41/meta/meta/classes.txt', 'r') as txt:
    classes = [l.strip() for l in txt.readlines()]
    class_to_ix = dict(zip(classes, range(len(classes))))
    ix_to_class = dict(zip(range(len(classes)), classes))
    class_to_ix = {v: k for k, v in ix_to_class.items()}
sorted_class_to_ix = collections.OrderedDict(sorted(class_to_ix.items()))

if not os.path.isdir('./test') and not os.path.isdir('./train'):

    def copytree(src, dst, symlinks = False, ignore = None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    copytree('../input/food41/images', './test', ignore=ignore_train)
    copytree('../input/food41/images', './train', ignore=ignore_test)
    
else:
    print('Train/Test files already copied into separate folders.')

../input/food41/images
../input/food41/images/lobster_roll_sandwich
../input/food41/images/chocolate_cake
../input/food41/images/baklava
../input/food41/images/croque_madame
../input/food41/images/poutine
../input/food41/images/pancakes
../input/food41/images/ramen
../input/food41/images/pho
../input/food41/images/crab_cakes
../input/food41/images/falafel
../input/food41/images/strawberry_shortcake
../input/food41/images/chicken_curry
../input/food41/images/carrot_cake
../input/food41/images/macarons
../input/food41/images/fish_and_chips
../input/food41/images/takoyaki
../input/food41/images/hamburger
../input/food41/images/hot_and_sour_soup
../input/food41/images/ravioli
../input/food41/images/sashimi
../input/food41/images/caesar_salad
../input/food41/images/omelette
../input/food41/images/cannoli
../input/food41/images/cup_cakes
../input/food41/images/bruschetta
../input/food41/images/edamame
../input/food41/images/clam_chowder
../input/food41/images/chicken_wings
../input/food41/im

In [7]:
data_dir = r'../input/food41/images'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       #transforms.RandomHorizontalFlip(),ImageNetPolicy(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(r'./train', transform=train_transforms)
test_data = datasets.ImageFolder(r'./test', transform=test_transforms)
#testdata= datasets.ImageFolder(data_dir + r'\test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
#test_loader=torch.utils.data.DataLoader(testdata, batch_size=64)

In [8]:
#defining the models
model=models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


In [9]:
import time
for device in ['cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")
    

Device = cuda; Time per batch: 0.053 seconds


In [10]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [11]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = models.densenet201(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(nn.Linear(1024,512),
                                 nn.LeakyReLU(),
                                 nn.Linear(512,256),
                                 nn.LeakyReLU(),
                                 nn.Linear(256,101))

criterion = nn.CrossEntropyLoss()

# Only train the classifier parameters, feature parameters are frozen
#optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [12]:
#using the Adam optimiser
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001, betas=[0.9, 0.999])

In [13]:
import numpy as np
import time
def train(n_epochs,trainloader,testloader, resnet, optimizer, criterion, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    running_loss=0
    
  
    for epoch in range(n_epochs):
        
        
        for inputs, labels in trainloader:
            
        # Move input and label tensors to the default device
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            start = time.time()
            logps = resnet(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
        
        
        resnet.eval()
        valid_loss=0
        accuracy=0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.cuda(), labels.cuda()
                logps = resnet(inputs)
                batch_loss = criterion(logps, labels)
                valid_loss += batch_loss.item()
                    
                    # Calculate accuracy
                
                top_p, top_class = logps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
           
        
            if valid_loss <= valid_loss_min:
                print("Validation loss decreased  Saving model")
                torch.save(resnet.state_dict(),'./food_classifier_densenet121_noise.pt')
                valid_loss_min=valid_loss
            
            print(f"Device = cuda; Time per batch: {(time.time() - start):.3f} seconds")       
            print(f"Epoch /{n_epochs}.. "
                  f"Train loss: {running_loss/len(trainloader):.3f}.. "
                  f"Test loss: {valid_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            resnet.train()

In [14]:
#training model for 50 epochs 
#accuracy obtained 64.1%
train(50,trainloader,testloader, model, optimizer, criterion,'./densenet.pt')

Validation loss decreased  Saving model
Device = cuda; Time per batch: 255.939 seconds
Epoch /50.. Train loss: 2.869.. Test loss: 1.858.. Test accuracy: 0.510
Validation loss decreased  Saving model
Device = cuda; Time per batch: 232.053 seconds
Epoch /50.. Train loss: 2.312.. Test loss: 1.688.. Test accuracy: 0.552
Validation loss decreased  Saving model
Device = cuda; Time per batch: 231.819 seconds
Epoch /50.. Train loss: 2.178.. Test loss: 1.523.. Test accuracy: 0.594
Validation loss decreased  Saving model
Device = cuda; Time per batch: 230.314 seconds
Epoch /50.. Train loss: 2.115.. Test loss: 1.505.. Test accuracy: 0.602
Validation loss decreased  Saving model
Device = cuda; Time per batch: 239.129 seconds
Epoch /50.. Train loss: 2.057.. Test loss: 1.469.. Test accuracy: 0.606
Validation loss decreased  Saving model
Device = cuda; Time per batch: 246.506 seconds
Epoch /50.. Train loss: 2.032.. Test loss: 1.460.. Test accuracy: 0.611
Validation loss decreased  Saving model
Device